In [ ]:
!wget https://hf.co/danjacobellis/walloc/resolve/main/RGB_Li_192c_J4_nf8_v1.0.2.pth

--2024-09-15 19:23:57--  https://hf.co/danjacobellis/walloc/resolve/main/RGB_Li_192c_J4_nf8_v1.0.2.pth
Resolving hf.co (hf.co)... 44.212.132.255, 34.204.155.59, 34.198.14.237, ...
Connecting to hf.co (hf.co)|44.212.132.255|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://huggingface.co/danjacobellis/walloc/resolve/main/RGB_Li_192c_J4_nf8_v1.0.2.pth [following]
--2024-09-15 19:23:57--  https://huggingface.co/danjacobellis/walloc/resolve/main/RGB_Li_192c_J4_nf8_v1.0.2.pth
Resolving huggingface.co (huggingface.co)... 99.86.102.9, 99.86.102.115, 99.86.102.128, ...
Connecting to huggingface.co (huggingface.co)|99.86.102.9|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/f3/ba/f3ba06623731c38247420d7019770a013d176a409abc994df4fee215d214a026/3368b9b93456413a5cb3e125dbdcd9284de70a890543b4dfcf148464df3d517f?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27RGB_Li

In [2]:
import io
import time
import torch
import numpy as np
import PIL
from torchvision.transforms import ToPILImage, PILToTensor
from datasets import load_dataset, Image
from walloc import walloc

class Config: pass

In [3]:
checkpoint = torch.load("RGB_Li_192c_J4_nf8_v1.0.2.pth",map_location="cpu",weights_only=False)
codec_config = checkpoint['config']
codec = walloc.Codec2D(
    channels = codec_config.channels,
    J = codec_config.J,
    Ne = codec_config.Ne,
    Nd = codec_config.Nd,
    latent_dim = codec_config.latent_dim,
    latent_bits = codec_config.latent_bits,
    lightweight_encode = codec_config.lightweight_encode
)
codec.load_state_dict(checkpoint['model_state_dict'])
codec.eval();

In [4]:
dataset = load_dataset("danjacobellis/LSDIR_val", split='validation')

Generating validation split:   0%|          | 0/250 [00:00<?, ? examples/s]

In [9]:
def walloc_compress(sample):
    with torch.no_grad():
        img = sample['image']
        x = PILToTensor()(img).to(torch.float)
        x = (x/255 - 0.5).unsqueeze(0).to(device)
        H, W = x.size(2), x.size(3)
        x_padded = walloc.pad(x,p=8)

        t0 = time.time()
        X = codec.wavelet_analysis(x_padded,codec.J)
        Y = codec.encoder(X)
        webp = walloc.latent_to_pil(Y.to("cpu"),codec.latent_bits, 3)[0]
        buff = io.BytesIO()
        webp.save(buff, format='WEBP', lossless=True)
        webp_bytes = buff.getbuffer()
        encode_time = time.time() - t0
        
        bpp = 8*len(webp_bytes)/(H*W)  
        
    return {
        'compressed': webp_bytes,
        'encode_time': encode_time,
        'bpp': bpp,
    }

In [ ]:
device = "cpu"
codec = codec.to(device)
walloc_ds = dataset.map(walloc_compress)
walloc_ds = walloc_ds.cast_column('compressed',Image())

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

In [ ]:
walloc_ds[28]['compressed']